#### Transacciones PDF 

In [37]:
pip install pymupdf pdfplumber tabula-py camelot-py[cv] pytesseract Pillow pandas camelot

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [38]:
# Lectura de pdf
import fitz  
import pdfplumber
import tabula
import pytesseract
from PIL import Image
import pandas as pd
import io
import re
import pandas as pd

In [39]:


class PDFExtractorAdaptativo:
    def __init__(self):
        self.metodos_texto = ['pdfplumber', 'pymupdf', 'ocr']
        self.metodos_tablas = ['tabula', 'pdfplumber']  # Quitamos 'camelot'
    
    def detectar_tipo_contenido(self, archivo_pdf):
        """Detecta qué tipo de contenido tiene el PDF"""
        try:
            doc = fitz.open(archivo_pdf)
            primera_pagina = doc[0]
            
            texto = primera_pagina.get_text().strip()
            imagenes = primera_pagina.get_images()
            
            with pdfplumber.open(archivo_pdf) as pdf:
                page = pdf.pages[0]
                tablas_detectadas = page.find_tables()
            
            resultado = {
                'tiene_texto': len(texto) > 50,
                'tiene_imagenes': len(imagenes) > 0,
                'posibles_tablas': len(tablas_detectadas) > 0,
                'es_escaneado': len(texto) < 50 and len(imagenes) > 0,
                'calidad_texto': 'alta' if len(texto) > 200 else 'baja'
            }
            
            doc.close()
            return resultado
            
        except Exception as e:
            return {'error': str(e)}
    
    def extraer_texto_robusto(self, archivo_pdf):
        """Extrae texto probando diferentes métodos"""
        metodos_resultados = {}
        
        # pdfplumber
        try:
            with pdfplumber.open(archivo_pdf) as pdf:
                texto = ""
                for page in pdf.pages:
                    texto += page.extract_text() + "\n"
                metodos_resultados['pdfplumber'] = texto
        except Exception as e:
            metodos_resultados['pdfplumber'] = f"Error: {e}"
        
        # pymupdf
        try:
            doc = fitz.open(archivo_pdf)
            texto = ""
            for page in doc:
                texto += page.get_text() + "\n"
            doc.close()
            metodos_resultados['pymupdf'] = texto
        except Exception as e:
            metodos_resultados['pymupdf'] = f"Error: {e}"
        
        # OCR
        try:
            doc = fitz.open(archivo_pdf)
            texto = ""
            for page_num in range(len(doc)):
                page = doc.load_page(page_num)
                pix = page.get_pixmap()
                img_data = pix.tobytes("png")
                img = Image.open(io.BytesIO(img_data))
                texto += pytesseract.image_to_string(img, lang='spa') + "\n"
            doc.close()
            metodos_resultados['ocr'] = texto
        except Exception as e:
            metodos_resultados['ocr'] = f"Error: {e}"
        
        # Seleccionar mejor resultado
        mejor_resultado = ""
        max_longitud = 0
        for metodo, resultado in metodos_resultados.items():
            if not resultado.startswith("Error") and len(resultado) > max_longitud:
                mejor_resultado = resultado
                max_longitud = len(resultado)
        
        return {
            'texto_final': mejor_resultado,
            'metodos_intentados': metodos_resultados,
            'metodo_exitoso': max([k for k, v in metodos_resultados.items() 
                                 if not v.startswith("Error") and len(v) == max_longitud], 
                                default="ninguno")
        }
    
    def extraer_tablas_robusto(self, archivo_pdf):
        """Extrae tablas probando diferentes métodos (sin camelot)"""
        resultados_tablas = {}
        
        # tabula-py
        try:
            tablas = tabula.read_pdf(archivo_pdf, pages='all', multiple_tables=True)
            resultados_tablas['tabula'] = {
                'tablas': tablas,
                'cantidad': len(tablas),
                'exito': True
            }
        except Exception as e:
            resultados_tablas['tabula'] = {'error': str(e), 'exito': False}
        
        # pdfplumber
        try:
            with pdfplumber.open(archivo_pdf) as pdf:
                todas_tablas = []
                for page in pdf.pages:
                    tablas_pagina = page.extract_tables()
                    for tabla in tablas_pagina:
                        df = pd.DataFrame(tabla[1:], columns=tabla[0])
                        todas_tablas.append(df)
                
                resultados_tablas['pdfplumber'] = {
                    'tablas': todas_tablas,
                    'cantidad': len(todas_tablas),
                    'exito': True
                }
        except Exception as e:
            resultados_tablas['pdfplumber'] = {'error': str(e), 'exito': False}
        
        return resultados_tablas
    
    def procesar_pdf_completo(self, archivo_pdf):
        """Proceso completo adaptativo"""
        print(f"Analizando: {archivo_pdf}")
        
        info_contenido = self.detectar_tipo_contenido(archivo_pdf)
        print(f"Análisis de contenido: {info_contenido}")
        
        resultado_final = {
            'archivo': archivo_pdf,
            'analisis_contenido': info_contenido,
            'texto': None,
            'tablas': None
        }
        
        if info_contenido.get('tiene_texto') or info_contenido.get('es_escaneado'):
            print("Extrayendo texto...")
            resultado_texto = self.extraer_texto_robusto(archivo_pdf)
            resultado_final['texto'] = resultado_texto
        
        if info_contenido.get('posibles_tablas'):
            print("Extrayendo tablas...")
            resultado_tablas = self.extraer_tablas_robusto(archivo_pdf)
            resultado_final['tablas'] = resultado_tablas
        
        return resultado_final

# Ejemplo de uso
if __name__ == "__main__":
    extractor = PDFExtractorAdaptativo()
    resultado = extractor.procesar_pdf_completo("resOurces/TADA.pdf")
    print("Resultado final:")
    print(resultado)


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


Analizando: resOurces/TADA.pdf
Análisis de contenido: {'tiene_texto': True, 'tiene_imagenes': True, 'posibles_tablas': True, 'es_escaneado': False, 'calidad_texto': 'alta'}
Extrayendo texto...
Extrayendo tablas...


CropBox missing from /Page, defaulting to MediaBox


Resultado final:
{'archivo': 'resOurces/TADA.pdf', 'analisis_contenido': {'tiene_texto': True, 'tiene_imagenes': True, 'posibles_tablas': True, 'es_escaneado': False, 'calidad_texto': 'alta'}, 'texto': {'texto_final': 'Orden de Compra : 297-00004261\nFriday, June 28, 2024\nPágina : 1\nZX VENTURES COLOMBIA\xa0\nN.I.T : 830094751\nCiudad : SANTAFE DE BOGOTA\nDirección : Carrera 53A No. 127 - 35\nTeléfono : 3204350538\nComprador :\nFax :\nPunto de Venta : Calle 18 #69f-35 TIENDAS EXPRESS\nCiudad : SANTAFE DE BOGOTA\nSitio de Entrega : Calle 18 #69f-35 TIENDAS EXPRESS\nCiudad : SANTAFE DE BOGOTA\nSitio de Entrega (Factura) : Calle 18 #69f-35 TIENDAS EXPRESS\nCiudad : SANTAFE DE BOGOTA\nEntidad a Facturar : 7707358310012 \xa0\xa0\xa0 ZX VENTURES COLOMBIA\xa0\nCiudad : SOCHA\nFecha mín. entrega :\nJun 28, 2024\nFecha máx. entrega :\nJun 28, 2024\nFecha Sol. entrega :\n-\nDATOS DEL PROVEEDOR : Croc S.A.S\nContacto : Leonilde Aguilar\nEAN : 7709997003799\nCiudad : SANTAFE DE BOGOTA\nCodigo loc

In [ ]:
#funcion de limpieza de columnas : 
def limpiar_columnas(df):
    df.columns = (
        df.columns
        .str.strip()                      
        .str.lower()                     
        .str.replace(' ', '_')        
        .str.replace(r'[^\w\s]', '', regex=True)  
    )
    return df

In [40]:
#extraer del diccionario final tablas y texto
textos = resultado.get('texto', {}).get('texto_final', '')
print(type(textos))

#Escritura .txt
with open("TADA.txt", "w", encoding="utf-8") as f:
    f.write(textos)

<class 'str'>


In [ ]:
#Procesamiento 
# ------------------------------
# EXTRACCIÓN DE DATOS GENERALES
# ------------------------------
datos_generales = {
    "Orden de Compra": re.search(r"Orden de Compra\s*:\s*(\S+)", textos).group(1),
    "Fecha de Orden": re.search(r"Friday,\s*(\w+ \d{1,2}, \d{4})", textos).group(1),
    "NIT": re.search(r"N\.I\.T\s*:\s*(\d+)", textos).group(1),
    "Ciudad Compañía": re.search(r"N\.I\.T\s*:\s*\d+\s*\nCiudad\s*:\s*(.+)", textos).group(1),
    "Dirección Compañía": re.search(r"Dirección\s*:\s*(.+)", textos).group(1),
    "Teléfono Compañía": re.search(r"Teléfono\s*:\s*(\d+)", textos).group(1),
    "Punto de Venta": re.search(r"Punto de Venta\s*:\s*(.+)", textos).group(1),
    "Ciudad Punto de Venta": re.findall(r"Punto de Venta\s*:.+\nCiudad\s*:\s*(.+)", textos)[0],
    "Sitio de Entrega": re.search(r"Sitio de Entrega\s*:\s*(.+)", textos).group(1),
    "Ciudad Entrega": re.findall(r"Sitio de Entrega\s*:.+\nCiudad\s*:\s*(.+)", textos)[0],
    "Entidad a Facturar": re.search(r"Entidad a Facturar\s*:\s*(\d+)", textos).group(1),
    "Ciudad Facturación": re.search(r"Entidad a Facturar\s*:.+\nCiudad\s*:\s*(.+)", textos).group(1),
    "Fecha mín. entrega": re.search(r"Fecha mín\. entrega\s*:\s*\n(.+)", textos).group(1),
    "Fecha máx. entrega": re.search(r"Fecha máx\. entrega\s*:\s*\n(.+)", textos).group(1),
    "Fecha Sol. entrega": re.search(r"Fecha Sol\. entrega\s*:\s*\n(.+)", textos).group(1),
    "Proveedor": re.search(r"DATOS DEL PROVEEDOR\s*:\s*(.+)", textos).group(1),
    "Contacto Proveedor": re.search(r"Contacto\s*:\s*(.+)", textos).group(1),
    "Teléfono Proveedor": re.search(r"Teléfono\s*:\s*(\d+)", textos).group(1),
    "Dirección Proveedor": re.search(r"Dirección\s*:\s*(.+)", textos).group(1),
}

In [46]:
datos_generales

{'Orden de Compra': '297-00004261',
 'Fecha de Orden': 'June 28, 2024',
 'NIT': '830094751',
 'Ciudad Compañía': 'SANTAFE DE BOGOTA',
 'Dirección Compañía': 'Carrera 53A No. 127 - 35',
 'Teléfono Compañía': '3204350538',
 'Punto de Venta': 'Calle 18 #69f-35 TIENDAS EXPRESS',
 'Ciudad Punto de Venta': 'SANTAFE DE BOGOTA',
 'Sitio de Entrega': 'Calle 18 #69f-35 TIENDAS EXPRESS',
 'Ciudad Entrega': 'SANTAFE DE BOGOTA',
 'Entidad a Facturar': '7707358310012',
 'Ciudad Facturación': 'SOCHA',
 'Fecha mín. entrega': 'Jun 28, 2024',
 'Fecha máx. entrega': 'Jun 28, 2024',
 'Fecha Sol. entrega': '-',
 'Proveedor': 'Croc S.A.S',
 'Contacto Proveedor': 'Leonilde Aguilar',
 'Teléfono Proveedor': '3204350538',
 'Dirección Proveedor': 'Carrera 53A No. 127 - 35'}

In [53]:
df_datos_generales = pd.DataFrame([datos_generales])
df_datos_generales  = limpiar_columnas(df_datos_generales )

In [54]:
df_datos_generales

,orden_de_compra,fecha_de_orden,nit,ciudad_compañía,dirección_compañía,teléfono_compañía,punto_de_venta,ciudad_punto_de_venta,sitio_de_entrega,ciudad_entrega,entidad_a_facturar,ciudad_facturación,fecha_mín_entrega,fecha_máx_entrega,fecha_sol_entrega,proveedor,contacto_proveedor,teléfono_proveedor,dirección_proveedor
0,297-00004261,"June 28, 2024",830094751,SANTAFE DE BOGOTA,Carrera 53A No. 127 - 35,3204350538,Calle 18 #69f-35 TIENDAS EXPRESS,SANTAFE DE BOGOTA,Calle 18 #69f-35 TIENDAS EXPRESS,SANTAFE DE BOGOTA,7707358310012,SOCHA,"Jun 28, 2024","Jun 28, 2024",-,Croc S.A.S,Leonilde Aguilar,3204350538,Carrera 53A No. 127 - 35


In [43]:
# ------------------------------
# EXTRACCIÓN DE LA TABLA DE PRODUCTOS
# ------------------------------
tabla_bruta = textos.split("Detalle de la Orden de Compra")[-1]
lineas = tabla_bruta.strip().splitlines()
lineas = [line.strip() for line in lineas if line.strip() != ""]

In [44]:
# ------------------------------
# CONVERSION DE PRODUCTOS
# ------------------------------
productos = []
i = 0
while i < len(lineas):
    if re.match(r"\d{13}", lineas[i]):  # EAN detectado
        try:
            producto = {
                "EAN": lineas[i],
                "Descripción": lineas[i+1],
                "Sub": lineas[i+2],
                "Precio Unitario": lineas[i+3].replace(",", ""),
                "Precio Neto": lineas[i+4].replace(",", ""),
                "Cantidad Pedida": lineas[i+5],
                "Unidad Medida": lineas[i+6],
                "Total Impuesto": lineas[i+7].replace(",", ""),
                "Subtotal sin Impuestos": lineas[i+8].replace(",", "")
            }
            productos.append(producto)
            i += 9
        except IndexError:
            break
    else:
        i += 1

df_productos = pd.DataFrame(productos)

In [51]:
df_productos = limpiar_columnas(df_productos)

In [52]:
df_productos

,ean,descripción,sub,precio_unitario,precio_neto,cantidad_pedida,unidad_medida,total_impuesto,subtotal_sin_impuestos
0,7709990234077,Espirale x 21 gr sabor Natural DON,1,1709,1709,180,NAR,61524,307620
1,7707369359819,ESPIRAL BBQ DON CHICHARRON,0,1709,1709,210,NAR,71778,358890
2,7707369351257,Espiral x 21 gr Sabor Chile Chipotl,0,1709,1709,90,NAR,30762,153810
